<a href="https://colab.research.google.com/github/Utkarshmishra2k2/AutoEstimator--Predict_Your_Car-s_Worth/blob/main/Tensflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Jay Mahakal")

In [ ]:
import math
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import shap

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

In [ ]:
import kagglehub
path = kagglehub.dataset_download("avikasliwal/used-cars-price-prediction")
train_01 = pd.read_csv(path + "/train-data.csv")
test_01 = pd.read_csv(path + "/test-data.csv")

In [ ]:
train_01 = train_01.drop(['Unnamed: 0' , 'New_Price' ], axis = 1)
test_01 = test_01.drop(['Unnamed: 0' , 'New_Price' ], axis = 1)

In [ ]:
print(train_01.shape)
print(train_01.info())
print(test_01.shape)
print(test_01.info())

In [ ]:
train_01.sample(5)

In [ ]:
test_01.sample(5)

In [ ]:
print(train_01.shape, test_01.shape)

In [ ]:
print(train_01.shape, test_01.shape)

In [ ]:
print(train_01.isnull().sum())

In [ ]:
print(test_01.isnull().sum())

In [ ]:
train_01["Mileage"] = train_01["Mileage"].str.replace(" km/kg", "")
train_01["Mileage"] = train_01["Mileage"].str.replace(" kmpl", "")

In [ ]:
train_01["Mileage"] = pd.to_numeric(train_01["Mileage"])

In [ ]:
train_01['Mileage'].replace(0.0,np.nan,inplace=True)

In [ ]:
print("Mean of Mileage =",np.mean(train_01['Mileage']))

In [ ]:
train_01['Mileage'].fillna(value=np.mean(train_01['Mileage']),inplace=True)

In [ ]:
train_01["Engine"] = train_01["Engine"].str.replace(" CC", "")

In [ ]:
train_01["Engine"] = pd.to_numeric(train_01["Engine"])

In [ ]:
train_01['Engine'].replace(0.0,np.nan,inplace=True)

In [ ]:
print("Mean of Engine =",np.mean(train_01['Engine']))

In [ ]:
train_01['Engine'].fillna(value=np.mean(train_01['Engine']),inplace=True)

In [ ]:
train_01["Power"] = train_01["Power"].str.replace(" bhp", "")

In [ ]:
train_01["Power"] = train_01["Power"].str.replace("null", "0.0")

In [ ]:
train_01["Power"] = pd.to_numeric(train_01["Power"])

In [ ]:
train_01['Power'].replace(0.0,np.nan,inplace=True)

In [ ]:
print("Mean of Power =",np.mean(train_01['Power']))

In [ ]:
train_01['Power'].fillna(value=np.mean(train_01['Power']),inplace=True)

In [ ]:
train_01['Seats'].replace(0.0,np.nan,inplace=True)

In [ ]:
print("Mean of Seats = ",np.mean(train_01['Seats']))

In [ ]:
train_01['Seats'].fillna(value=np.mean(train_01['Seats']),inplace=True)

In [ ]:
print(train_01.isnull().sum())

In [ ]:
train_01.sample(5)

In [ ]:
train_01.describe()

In [ ]:
train_01.info()

In [ ]:
train_02 = train_01.copy()
train_02['Brand'] = train_02.Name.str.split().str.get(0)
train_02.sample(5)

In [ ]:
print(train_01["Location"].nunique())
print(train_01["Location"].unique())

In [ ]:
fig = px.bar(train_01, x=train_01['Location'].value_counts().index,
             y=train_01['Location'].value_counts().values,
             title='Distribution of Locations',
             labels={'x': 'Location', 'y': 'Count'},
             color=train_01['Location'].value_counts().index,
             color_discrete_sequence=px.colors.qualitative.Set3)

fig.update_layout(
    xaxis_title='Location',
    yaxis_title='Count',
    template='plotly_dark',
    xaxis_tickangle=45,
    bargap=0.1
)
fig.update_traces(text=train_01['Location'].value_counts().values,
                  textposition='outside',
                  texttemplate='%{text}')
fig.show()

**Overall Distribution:**

The distribution of locations appears to be skewed to the right, with a few locations having significantly higher counts than others.


**Top Locations:**

* Mumbai has the highest count of locations, followed by Hyderabad and Kochi.
The top 3 locations (Mumbai, Hyderabad, and Kochi) account for a significant portion of the total count.

**Mid-Range Locations:**

* There is a cluster of locations with counts in the 500-600 range, including Coimbatore, Pune, Delhi, and Kolkata.

**Lower-End Locations:**

* Chennai, Jaipur, Bangalore, and Ahmedabad have the lowest counts.

In [ ]:
fig = px.pie(train_01, names=train_01['Location'].value_counts().index,
             values=train_01['Location'].value_counts().values,
             title='Distribution of Locations',
             color=train_01['Location'].value_counts().index,
             color_discrete_sequence=px.colors.qualitative.Set3,
             template='plotly_dark',
             hole=0.3)

fig.update_traces(textinfo='percent+label', pull=[0.1 if x == 'Top Location' else 0 for x in train_01['Location'].value_counts().index])
fig.show()


**Overall Distribution:**

* The data is distributed across 12 categories (locations).
There is a wide range of percentages, indicating varying levels of representation for each location.

**Top Locations:**

* Mumbai and Hyderabad have the highest representation, with 13.1% and 12.3%, respectively.
The top 3 locations (Mumbai, Hyderabad, and Kochi) account for approximately 36% of the total.

**Mid-Range Locations:**

*Several locations fall in the mid-range, including Coimbatore, Pune, Delhi, Kolkata, Chennai, and Bangalore, with percentages ranging from 8.09% to 10.8%.

**Lower-End Locations:**

* Ahmedabad and Jaipur have the lowest representation, with 3.72% and 6.06%, respectively.

***Potential Insights:***

1. Regional Concentration: The higher percentages for Mumbai and Hyderabad might suggest a concentration of activity or interest in these regions.

2. Diversity: The presence of multiple locations with similar percentages indicates a diverse distribution across different regions.

In [ ]:
train_01["Year"].unique()

In [ ]:
fig = px.bar(train_01, x=train_01['Year'].value_counts().index,
             y=train_01['Year'].value_counts().values,
             title='Distribution of Years',
             labels={'x': 'Year', 'y': 'Count'},
             color=train_01['Year'].value_counts().index,
             color_discrete_sequence=px.colors.qualitative.Set1)

fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Count',
    xaxis_tickangle=45,
    template='plotly_dark',
    bargap=0.1,
    showlegend=True
)


fig.update_traces(text=train_01['Year'].value_counts().values,
                  textposition='outside',
                  texttemplate='%{text}')

fig.show()


**Overall Distribution:**

* The distribution of data points across the years is not uniform. There is a clear upward trend, indicating an increase in the number of observations over time.
* The distribution appears to be skewed to the right, with a few years having significantly higher counts than others.

**Top Year:**

* 2015 has the highest count, followed by 2010 and 2005.

**Lower-End Years:**

* 2000 has the lowest count, followed by 1998 and 1995.

***Potential Insights:***

1. Growth Trend: The increasing trend in the number of observations suggests a growing dataset or increasing activity over time.

2. Outliers: The significantly lower counts for the earlier years (1995, 1998, and 2000) could be due to various factors, such as data collection limitations or changes in the system being observed.

In [ ]:
percentile_85 = train_01['Kilometers_Driven'].quantile(0.99)
train_filtered = train_01[train_01['Kilometers_Driven'] <= percentile_85]
fig = px.histogram(train_filtered, x='Kilometers_Driven', nbins=40,
                   title='Distribution of Kilometers Driven',
                   labels={'Kilometers_Driven': 'Kilometers Driven'},
                   color_discrete_sequence=['maroon'])

fig.update_layout(
    title_text='Kilometers Driven Distribution',
    title_x=0.5,
    title_font_size=20,
    xaxis_title='Kilometers Driven',
    yaxis_title='Frequency',
    template='plotly_dark',
    bargap=0.2,
    xaxis=dict(showgrid=True, zeroline=False),
    yaxis=dict(showgrid=True, zeroline=False),
)

fig.show()

**Overall Distribution:**

* The distribution of kilometers driven appears to be right-skewed. This means that there are more vehicles with lower mileage and fewer vehicles with higher mileage.
The majority of vehicles seem to have driven between 20k and 80k kilometers.

**Peak Mileage:**

* The highest frequency of vehicles is observed in the range of 60k-80k kilometers.

**Tailing Off:**

* As the mileage increases beyond 80k kilometers, the frequency of vehicles starts to decline rapidly.

***Potential Insights:***

1. Vehicle Age: The right-skewed distribution might suggest that the dataset includes a mix of newer and older vehicles. Newer vehicles would generally have lower mileage.
2. Usage Patterns: The peak frequency around 60k-80k kilometers could indicate a common usage pattern, such as vehicles being driven for a certain period before being replaced or sold.

In [ ]:
fig = px.box(train_filtered, x='Kilometers_Driven', title='Boxplot of Kilometers Driven')

fig.update_layout(
    xaxis_title='Kilometers Driven',
    yaxis_title='Value',
    title_x=0.5,
    title_font=dict(size=20, family='Arial', color='darkblue'),
    template='plotly_dark'
)

fig.show()


**Overall Distribution:**

The distribution of kilometers driven appears to be right-skewed, as indicated by the long whisker on the right side of the boxplot. This means that there are more vehicles with lower mileage and fewer vehicles with higher mileage.

**Central Tendency:**

The median (the line within the box) appears to be around 60,000 kilometers. This suggests that half of the vehicles have driven less than 60,000 kilometers, and half have driven more.
The box itself represents the interquartile range (IQR), which is the range of the middle 50% of the data. In this case, the IQR seems to be between approximately 40,000 and 70,000 kilometers.

**Outliers:**

The dots on the right side of the plot represent outliers, which are data points that fall significantly outside the range of the majority of the data. In this case, there are several outliers with high mileage, further emphasizing the right-skewed nature of the distribution.

***Potential Insights:***

1. Vehicle Age: The right-skewed distribution and presence of outliers with high mileage might suggest that the dataset includes a mix of newer and older vehicles. Newer vehicles would generally have lower mileage.
2. Usage Patterns: The median and IQR give us an idea of the typical range of mileage for most vehicles in the dataset. The outliers with high mileage might represent vehicles that are used extensively or for specific purposes.

In [ ]:
fuel_counts = train_01['Fuel_Type'].value_counts().reset_index()
fuel_counts.columns = ['Fuel_Type', 'Count']

fig = px.bar(fuel_counts,
             x='Count',
             y='Fuel_Type',
             orientation='h',
             title='Fuel Type Distribution',
             labels={'Fuel_Type': 'Fuel Type', 'Count': 'Number of Vehicles'},
             color='Fuel_Type',
             color_continuous_scale='Viridis',
             template='plotly_dark'
            )

fig.update_layout(
    xaxis_title="Number of Vehicles",
    yaxis_title="Fuel Type",
    title_x=0.5,
    title_font=dict(size=20, family='Arial', color='white'),
    paper_bgcolor='rgb(17,17,17)',
    plot_bgcolor='rgb(17,17,17)',
    font=dict(color='white')  )

fig.show()


**Overall Distribution:**

* The distribution of vehicles across different fuel types is heavily skewed towards diesel and petrol, with a significantly lower number of vehicles using CNG, LPG, and electric fuel types.

**Dominant Fuel Types:**

* Diesel and petrol are the most common fuel types, with diesel having a slightly higher number of vehicles than petrol.

**Less Common Fuel Types:**

* CNG, LPG, and electric vehicles have a significantly lower number of vehicles compared to diesel and petrol.

In [ ]:
owner_counts = train_01['Owner_Type'].value_counts().reset_index()
owner_counts.columns = ['Owner_Type', 'Count']
fig = px.pie(owner_counts, names='Owner_Type',
             values='Count',
             title='Distribution of Owner Types',
             color='Owner_Type',
             color_discrete_sequence=px.colors.qualitative.Set3,
             template='plotly_dark',
             hole=0.3)

fig.update_traces(textinfo='percent+label', pull=[0.1 if x == 'Top Location' else 0 for x in train_01['Location'].value_counts().index])
fig.show()


**Overall Distribution:**

* The distribution of owner types is heavily skewed towards the "First" owner category, with a significantly lower number of vehicles in the other categories.
Dominant Owner Type:

* "First" owners represent the vast majority of vehicles, accounting for approximately 81.9% of the total.

**Less Common Owner Types:**

* "Second" owners make up the next largest group, with 16.1% of the vehicles.
"Third" and "Fourth & Above" owners are relatively rare, with 1.88% and 0.15%, respectively.

***Potential Insights:***

1. Market Dynamics: The dominance of "First" owners suggests that a large portion of vehicles are likely new purchases or owned by individuals who have kept their vehicles for a significant period.
2. Resale Market: The smaller percentages of "Second," "Third," and "Fourth & Above" owners might indicate a relatively active resale market, with vehicles changing hands multiple times.

In [ ]:
fig = px.histogram(train_01, x='Mileage', nbins=40,
                   title='Distribution of Mileage',
                   labels={'Mileage': 'Mileage'},
                   color_discrete_sequence=['maroon'])

fig.update_layout(
    title_text='Mileage Distribution',
    title_x=0.5,
    title_font_size=20,
    xaxis_title='Mileage',
    yaxis_title='Frequency',
    template='plotly_dark',
    bargap=0.2,
    xaxis=dict(showgrid=True, zeroline=False),
    yaxis=dict(showgrid=True, zeroline=False),
)

fig.show()

**Overall Distribution:**

* The distribution of mileage appears to be roughly bell-shaped or normal, with a central peak and tapering tails on both sides. This indicates that the majority of vehicles have a mileage within a certain range, with fewer vehicles having significantly lower or higher mileage.

**Central Tendency:**

* The peak of the distribution appears to be around 20, suggesting that this is the most common mileage range for vehicles in this dataset.

**Spread:**

* The distribution seems to be relatively narrow, with most of the data concentrated around the peak. This indicates that the vehicles in this dataset have a fairly consistent mileage range.

In [ ]:
fig = px.box(train_01, x='Mileage', title='Boxplot of Mileage')

fig.update_layout(
    xaxis_title='Mileage',
    yaxis_title='Value',
    title_x=0.5,
    title_font=dict(size=20, family='Arial', color='darkblue'),
    template='plotly_dark'
)

fig.show()


**Overall Distribution:**

* The distribution of mileage appears to be slightly right-skewed, as indicated by the longer whisker on the right side of the boxplot. This means that there are more vehicles with lower mileage and fewer vehicles with higher mileage.

**Central Tendency:**

* The median (the line within the box) appears to be around 17-18. This suggests that half of the vehicles have a mileage of less than 17-18, and half have a higher mileage.
* The box itself represents the interquartile range (IQR), which is the range of the middle 50% of the data. In this case, the IQR seems to be between approximately 15 and 20.

**Outliers:**

* The dots on the right side of the plot represent outliers, which are data points that fall significantly outside the range of the majority of the data. In this case, there are a few outliers with higher mileage, further emphasizing the right-skewed nature of the distribution.

In [ ]:
fig = px.histogram(train_01, x='Engine', nbins=40,
                   title='Distribution of Engine Sizes',
                   labels={'Mileage': 'Engine Sizes'},
                   color_discrete_sequence=['LightBlue'])

fig.update_layout(
    title_text='Engine Sizes Distribution',
    title_x=0.5,
    title_font_size=20,
    xaxis_title='Engine Sizes',
    yaxis_title='Frequency',
    template='plotly_dark',
    bargap=0.2,
    xaxis=dict(showgrid=True, zeroline=False),
    yaxis=dict(showgrid=True, zeroline=False),
)

fig.show()

**Overall Distribution:**

* The distribution of engine sizes appears to be right-skewed, with a long tail towards higher engine sizes. This means that there are more vehicles with smaller engines and fewer vehicles with larger engines.
* The distribution has multiple peaks, suggesting that there might be clusters of vehicles with specific engine sizes.

**Central Tendency:**

* The majority of vehicles seem to have engine sizes in the range of 1000-2000 cc. This is where the highest frequency is observed.

**Spread:**

* The distribution is relatively wide, indicating that there is a significant variation in engine sizes across the vehicles.

In [ ]:
fig = px.histogram(train_01, x='Power', nbins=40,
                   title='Distribution of Power ',
                   labels={'Power': 'Power '},
                   color_discrete_sequence=['Lightgreen'])

fig.update_layout(
    title_text='Power Distribution',
    title_x=0.5,
    title_font_size=20,
    xaxis_title='Power',
    yaxis_title='Frequency',
    template='plotly_dark',
    bargap=0.2,
    xaxis=dict(showgrid=True, zeroline=False),
    yaxis=dict(showgrid=True, zeroline=False),
)

fig.show()

In [ ]:
fig = px.histogram(train_01, x='Seats', nbins=40,
                   title='Distribution of Seats ',
                   labels={'Seats': 'Seats Coynts '},
                   color_discrete_sequence=['Lightgreen'])

fig.update_layout(
    title_text='Seats Distribution',
    title_x=0.5,
    title_font_size=20,
    xaxis_title='Seats',
    yaxis_title='Frequency',
    template='plotly_dark',
    bargap=0.2,
    xaxis=dict(showgrid=True, zeroline=False),
    yaxis=dict(showgrid=True, zeroline=False),
)

fig.show()

**Overall Distribution:**

* The distribution of power seems to be right-skewed, with a long tail towards higher power values. This means that there are more devices with lower power consumption and fewer devices with higher power consumption.
* The distribution has multiple peaks, suggesting that there might be clusters of devices with specific power consumption ranges.

**Central Tendency:**

* The majority of devices seem to have power consumption in the range of 50-100 watts. This is where the highest frequency is observed.

**Spread:**

* The distribution is relatively wide, indicating that there is a significant variation in power consumption across the devices.

In [ ]:
fig = px.box(train_01, x='Price', title='Boxplot of Car Prices')

fig.update_layout(
    xaxis_title='Price',
    yaxis_title='Value',
    title_x=0.5,
    title_font=dict(size=20, family='Arial', color='darkblue'),
    template='plotly_dark'
)

fig.show()


**Overall Distribution:**

* The distribution of prices appears to be heavily right-skewed, as indicated by the long whisker on the right side of the boxplot. This means that there are many products with lower prices and fewer products with very high prices.

**Central Tendency:**

* The median (the line within the box) appears to be around 15. This suggests that half of the products have a price of less than 15, and half have a higher price.
* The box itself represents the interquartile range (IQR), which is the range of the middle 50% of the data. In this case, the IQR seems to be quite narrow, indicating that the majority of products are priced within a relatively small range.

**Outliers:**

* The dots on the right side of the plot represent outliers, which are data points that fall significantly outside the range of the majority of the data.
* In this case, there are several outliers with very high prices, further emphasizing the right-skewed nature of the distribution.

In [ ]:
fig = px.violin(train_01,
                x='Price',
                box=True,
                points='all',
                title='Violin Plot of Car Prices')

fig.update_layout(
    title_x=0.5,
    title_font=dict(size=20, family='Arial', color='white'),
    yaxis_title="Price",
    paper_bgcolor='rgb(0, 0, 0)',
    plot_bgcolor='rgb(0, 0, 0)',
    font=dict(color='white')
)

fig.show()


**Overall Distribution:**

* The distribution of car prices is highly right-skewed, with a long tail towards higher prices. This indicates that most cars are priced relatively low, while a few high-end cars significantly increase the overall price range.
* The violin plot's shape suggests that the distribution might be multimodal, with potential clusters of prices at different levels.

**Central Tendency:**

* The median price, represented by the horizontal line within the box plot, appears to be around 15. This means that half of the cars are priced below 15, and half are priced higher.
* The box plot also shows the interquartile range (IQR), which is the range of the middle 50% of the data. In this case, the IQR seems to be quite narrow, indicating that the majority of cars are priced within a relatively small range.

**Outliers:**

* The numerous dots scattered along the x-axis represent individual data points that are considered outliers. These outliers are primarily located in the higher price range, further emphasizing the right-skewed nature of the distribution.

In [ ]:
avg_price_by_year = train_01.groupby('Year')['Price'].mean().reset_index()

fig = px.line(
    avg_price_by_year,
    x='Year',
    y='Price',
    title='Average Car Price by Year',
    labels={'Year': 'Car Year', 'Price': 'Average Price (in $)'},
    line_shape='linear' ,
    template='plotly_dark'
)

fig.show()

**Overall Trend:**

* The average car price has shown an overall increasing trend from 2000 to 2020. This indicates that the cost of cars has generally risen over this period.

**Specific Trends:**

* There are periods of more rapid price increases, such as between 2010 and 2015, and periods of slower growth, like between 2000 and 2005.
* There are also some fluctuations and temporary decreases in the average price, suggesting that factors like economic conditions or market trends can influence car prices.

***Potential Insights:***

1. Inflation and Economic Factors: The overall upward trend might be influenced by factors like inflation, economic growth, and increased consumer demand for cars.
2.  Technological Advancements: New technologies, features, and stricter safety regulations can increase the cost of manufacturing and thus the final price of cars.
3. Market Dynamics: Fluctuations in the market, such as changes in fuel prices, interest rates, or consumer preferences, can impact the average car price.

In [ ]:

fig = px.scatter_3d(
    train_01,
    x='Kilometers_Driven',
    y='Mileage',
    z='Price',
    color='Price',
    template="plotly_dark",
    title="3D Scatter Plot: Kilometers, Mileage vs Price",
    labels={'Kilometers_Driven': 'Kilometers Driven', 'Mileage': 'Mileage', 'Price': 'Price (in $)'}
)

fig.show()


**Overall Relationship:**

* There appears to be a weak negative correlation between Kilometers Driven and Price. This means that, generally, as the kilometers driven increase, the price tends to decrease. However, the relationship is not very strong, as there is significant scatter in the data points.

**Clusters and Outliers:**

* The plot shows clusters of points, indicating groups of cars with similar price and mileage combinations.
* There are also some outliers, which are points that deviate significantly from the general trend. These outliers might represent unique or rare cars with high prices and low mileage, or vice versa.

**Impact of Power:**

* The color of the bubbles represents the power of the car. It seems that cars with higher power tend to be more expensive, regardless of the mileage. However, there is still significant overlap between different power levels and prices.

In [ ]:
fig = px.scatter(
    train_01,
    x='Kilometers_Driven',
    y='Price',
    size='Power',
    color='Power',
    template="plotly_dark",
    title="Kilometers Driven vs Price (Bubble Plot)",
    labels={'Kilometers_Driven': 'Kilometers Driven', 'Price': 'Price (in $)', 'Power': 'Power'}
)

fig.show()

In [ ]:
columns_to_plot = ['Mileage', 'Power', 'Kilometers_Driven', 'Engine']
for column in columns_to_plot:
    fig = px.scatter(train_01, x=column, y='Price', trendline='ols',
                     title=f"{column} vs Price",
                     template="plotly_dark",
                     labels={column: f"{column}", 'Price': 'Price (in INR)'})

    fig.show()


**Mileage and Price:**
* A weak negative correlation exists, meaning as mileage increases, price tends to decrease. However, the relationship is not strong due to significant scatter and outliers.

**Power and Price:**
* A weak positive correlation shows that cars with higher power tend to have higher prices, but the relationship is weak, with significant variability.

**Kilometers Driven and Price:**
* There is a weak negative correlation, where higher kilometers driven generally results in lower prices, though the trend is not strong due to scattered data.

**Engine Size and Price:**
* A weak positive correlation indicates that larger engines tend to have higher prices, but the relationship is weak, with considerable scatter.

## MulitVarite Analysis

In [ ]:

fig = px.scatter_3d(
    train_01,
    x='Kilometers_Driven',
    y='Mileage',
    z='Price',
    color='Price',
    template="plotly_dark",
    title="3D Scatter Plot: Kilometers, Mileage vs Price",
    labels={'Kilometers_Driven': 'Kilometers Driven', 'Mileage': 'Mileage', 'Price': 'Price (in $)'}
)

fig.show()


**Overall Relationship:**

* The 3D plot reinforces the weak negative correlation between kilometers driven and price. However, it shows that price is also influenced by other factors, notably power.

**Clusters and Outliers:**

* Clusters of cars with similar price, mileage, and power combinations are visible.
* Outliers are present, representing cars with unusual combinations, such as low mileage but high prices or vice versa.

**Impact of Power:**

* Power is indicated by the color of the bubbles, and cars with higher power tend to have higher prices, regardless of their mileage or kilometers driven. However, there is still significant overlap, suggesting that power alone is not a strong predictor of price.

In [ ]:
fig_sunburst = px.sunburst(train_02, path=['Fuel_Type', 'Brand'], values='Mileage',
                           title="Fuel Type vs Brand Distribution of Mileage",
                           labels={'Fuel_Type': 'Fuel Type', 'Brand': 'Brand', 'Mileage': 'Mileage (km/l)'},
                           template='plotly_dark')

fig_sunburst.show()


**Overall Distribution:**

* The distribution of car brands is dominated by petrol and diesel fuel types.
* There is a significant difference in the number of cars using petrol and diesel fuel types.
* The number of cars using other fuel types (electric, CNG, LPG) is negligible.

**Brand-wise Distribution:**

* Petrol: Maruti, Hyundai, and Honda appear to be the most popular brands for petrol cars.
* Diesel: Mahindra, Ford, and Toyota seem to be the most popular brands for diesel cars.

***Potential Insights:***

1. Market Dominance: Maruti and Hyundai appear to have a significant market share in the petrol car segment.

In [ ]:
fig = px.box(train_01, x='Transmission', y='Mileage',
             title="Transmission vs Mileage",
             labels={'Transmission': 'Transmission Type', 'Mileage': 'Mileage (km/l)'},
             template='plotly_dark')

fig.update_layout(
    xaxis_title="Transmission Type",
    yaxis_title="Mileage (km/l)",
    title_x=0.5,
    title_font=dict(size=30),
    xaxis=dict(tickmode='array', tickvals=[0, 1], ticktext=['Manual', 'Automatic'])
)

fig.show()


**Overall Mileage:**

* Manual Transmission: The majority of cars with manual transmission have a mileage between 17 and 22 km/l. The median mileage for manual transmission cars is around 20 km/l.
* Automatic Transmission: The majority of cars with automatic transmission have a mileage between 15 and 19 km/l. The median mileage for automatic transmission cars is around 17 km/l.

**Variability:**

* Manual Transmission: The spread of mileage values for manual transmission cars is wider, with a range of approximately 15 to 32 km/l.
* Automatic Transmission: The spread of mileage values for automatic transmission cars is narrower, with a range of approximately 10 to 27 km/l.

**Outliers:**

* Manual Transmission: There is one outlier on the higher end of the mileage range for manual transmission cars.
* Automatic Transmission: There are two outliers on the higher end of the mileage range for automatic transmission cars.

**Comparison:**

* Median Mileage: Manual transmission cars generally have a higher median mileage compared to automatic transmission cars.
* Variability: Manual transmission cars exhibit higher variability in mileage compared to automatic transmission cars.

In [ ]:

fig = px.bar(train_02.groupby('Brand')['Power'].mean().reset_index(),
             x='Brand', y='Power',
             title="Brand vs Power",
             labels={'Brand': 'Brand', 'Power': 'Average Power (HP)'},
             color='Brand',
             template='plotly_dark')

fig.update_layout(
    xaxis_title="Brand",
    yaxis_title="Average Power (HP)",
    title_x=0.5,
    title_font=dict(size=30),
    xaxis_tickangle=-45,
)

fig.show()

**Overall Power:**

* The majority of car brands have an average power output between 100 and 200 HP.There are a few brands with average power output exceeding 300 HP.

**Variability:**

* There is a significant variation in average power across different car brands.
Some brands have a much higher average power output compared to others.

**Comparison:**

* Brands with the highest average power output include Lamborghini, Mahindra, and Maruti.
* Brands with the lowest average power output include Ambassador, Audi, and BMW.

In [ ]:
fig = px.histogram(
    train_01,
    x='Price',
    facet_col='Location',
    facet_row='Transmission',
    color='Fuel_Type',
    nbins=50,
    title="Car Price Distribution by Location, Transmission, and Fuel Type",
    template="plotly_dark",
    labels={
        'Price': 'Car Price (in $)',
        'Location': 'Car Location',
        'Transmission': 'Transmission Type',
        'Fuel_Type': 'Fuel Type',
        'Seats': 'Number of Seats',
        'Owner_Type': 'Owner Type',
        'Brand': 'Car Brand'
    }
)

fig.update_layout(
    title={'x': 0.5, 'xanchor': 'center'},
    margin=dict(t=50, b=50, l=50, r=50),
    showlegend=True,
    legend_title="Fuel Type",
)

fig.show()

**Overall Price Distribution:**

* The majority of car prices fall within the range of $0 to $100.
There are some car prices that exceed $150.

**Variability:**

* There is a significant variation in car prices across different locations, transmission types, and fuel types.
Some locations, transmission types, and fuel types have a higher number of cars with higher prices compared to others.

**Comparison:**

* Locations: Mumbai and Delhi have the highest number of cars with prices exceeding 150.
* Transmission Type: Manual transmission cars have a higher number of cars with prices exceeding Rs150 compared to automatic transmission cars.
* Fuel Type: CNG and Petrol cars have a higher number of cars with prices exceeding $150 compared to Diesel, LPG, and Electric cars.

In [ ]:
fig = px.parallel_categories(
    train_02,
    dimensions=['Location', 'Transmission', 'Fuel_Type', 'Seats', 'Owner_Type', 'Brand'],
    color='Price',
    color_continuous_scale='Viridis',
    title="Car Price Analysis Based on Features",
    template="plotly_dark",
    labels={
        'Location': 'Car Location',
        'Transmission': 'Transmission Type',
        'Fuel_Type': 'Fuel Type',
        'Seats': 'Number of Seats',
        'Owner_Type': 'Owner Type',
        'Brand': 'Car Brand',
        'Price': 'Price (in ₹)'
    }
)

fig.update_traces(
    hovertemplate="%{dimension}: %{label}<br>Price: %{color:.2f}<extra></extra>"  # Format hover data
)

fig.update_layout(
    title={'x': 0.5, 'xanchor': 'center'},
    margin=dict(t=50, b=50, l=50, r=50),
)

fig.show()

**Price Distribution:**

* The majority of car prices appear to fall within the range of 20 to 100.
* There are some car prices that exceed 140, with a few outliers reaching even higher price points.

**Variability:**

* There is a significant variation in car prices across different car brands.
* Some brands, such as Hyundai and Honda, have a higher number of cars with higher prices compared to others like Lamborghini and Ferrari.

**Feature Influence:**

* The chord diagram suggests that certain features might be associated with higher car prices. For example, cars from Mumbai or Delhi, with automatic transmission, petrol or CNG fuel, and a higher number of seats, seem to have a higher proportion of cars with higher prices.
* However, without more specific data and statistical analysis, it's difficult to quantify the exact impact of each feature on price.

In [ ]:
train_01['Brand'] = train_01.Name.str.split().str.get(0)
test_01['Brand'] = test_01.Name.str.split().str.get(0)

In [ ]:
train_01.drop(columns=['Name'], axis=1, inplace=True)
test_01.drop(columns=['Name'], axis=1, inplace=True)

In [ ]:
scaler = StandardScaler()
numeric_columns = test_01.select_dtypes(include=['number']).columns

train_01[numeric_columns] = scaler.fit_transform(train_01[numeric_columns])
test_01[numeric_columns] = scaler.fit_transform(test_01[numeric_columns])

In [ ]:
label_encoder = LabelEncoder()

cat_columns = test_01.select_dtypes(exclude=['number']).columns

for column in cat_columns:
    train_01[column] = label_encoder.fit_transform(train_01[column])
    test_01[column] = label_encoder.fit_transform(test_01[column])

In [ ]:
X = train_01.drop(columns = ["Price"])
Y = train_01["Price"]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(512, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))  # Dropout layer for regularization

In [ ]:
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))  # Dropout for regularization

In [ ]:
model.add(Dense(1))

In [ ]:
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

In [ ]:
history = model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_data=(X_test, Y_test), verbose=1)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(1, len(history.history['loss']) + 1)),
    y=history.history['loss'],
    mode='lines+markers',
    name='Train Loss',
    line=dict(color='blue'),
    marker=dict(size=6, symbol='circle', color='blue')
))

fig.add_trace(go.Scatter(
    x=list(range(1, len(history.history['val_loss']) + 1)),
    y=history.history['val_loss'],
    mode='lines+markers',
    name='Validation Loss',
    line=dict(color='red'),
    marker=dict(size=6, symbol='circle', color='red')
))

fig.update_layout(
    title='Training and Validation Loss Over Epochs',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    legend_title='Loss Type',
    template='plotly_dark',
    hovermode='closest'
)

fig.show()


In [ ]:
test_01

In [ ]:
y_pred_nn = model.predict(X_test)

In [ ]:
print(f"Neural Network - MAE: {mean_absolute_error(Y_test, y_pred_nn)}")
print(f"Neural Network - MSE: {mean_squared_error(Y_test, y_pred_nn)}")
print(f"Neural Network - RMSE: {np.sqrt(mean_squared_error(Y_test, y_pred_nn))}")
print(f"Neural Network - R² Score: {r2_score(Y_test, y_pred_nn)}")

In [ ]:
import joblib
joblib.dump(model, "tensflow.joblib")
